# ESA WorldCover - Extraction des Données d'Occupation du Sol

## C'est quoi ESA WorldCover ?

**ESA WorldCover** est une carte mondiale d'occupation du sol produite par l'Agence Spatiale Européenne :
- Résolution spatiale de **10 mètres** (très précise !)
- Basée sur les images **Sentinel-1 et Sentinel-2**
- **11 classes** d'occupation du sol

## Pourquoi c'est utile pour la qualité de l'eau ?

L'occupation du sol autour d'un point d'eau affecte directement sa qualité :

| Occupation du sol | Impact sur la qualité de l'eau |
|-------------------|--------------------------------|
| **Zones agricoles** | Ruissellement d'engrais → ↑ Phosphore |
| **Zones urbaines** | Pollution, ruissellement imperméable → ↑ Conductivité |
| **Forêts** | Filtration naturelle → ↓ Pollution |
| **Zones humides** | Absorption des nutriments → ↓ Phosphore |
| **Sol nu** | Érosion → ↑ Turbidité, sédiments |

## Ce que fait ce notebook

```
1. Se connecter à l'API Microsoft Planetary Computer
         ↓
2. Pour chaque site de mesure d'eau :
   - Définir un buffer de 500m autour du point
   - Télécharger la tuile ESA WorldCover correspondante
   - Calculer le pourcentage de chaque classe d'occupation
         ↓
3. Sauvegarder le fichier CSV avec les features d'occupation du sol
```

## Classes ESA WorldCover

| Code | Classe | Description |
|------|--------|-------------|
| 10 | Tree cover | Forêts, arbres |
| 20 | Shrubland | Arbustes |
| 30 | Grassland | Prairies, herbages |
| 40 | Cropland | Zones agricoles |
| 50 | Built-up | Zones urbaines |
| 60 | Bare / sparse vegetation | Sol nu, végétation clairsemée |
| 70 | Snow and ice | Neige et glace |
| 80 | Permanent water bodies | Plans d'eau permanents |
| 90 | Herbaceous wetland | Zones humides herbacées |
| 95 | Mangroves | Mangroves |
| 100 | Moss and lichen | Mousse et lichen |

## Source des données

Documentation : [ESA WorldCover sur Planetary Computer](https://planetarycomputer.microsoft.com/dataset/esa-worldcover)

---

## Étape 1 : Installation des dépendances

**Première exécution uniquement** : Après avoir exécuté cette cellule, il faut redémarrer le kernel :
1. Cliquer sur "Connected" en haut
2. Sélectionner "Restart kernel"

Les exécutions suivantes n'ont pas besoin de ce redémarrage.

In [1]:
!pip install uv
!uv pip install --system -r ../requirements.txt

Defaulting to user installation because normal site-packages is not writeable


[notice] A new release of pip is available: 24.3.1 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Using Python 3.13.1 environment at: c:\Program Files\Python313
Resolved 194 packages in 4.33s
error: Failed to install: jupyter_events-0.12.0-py3-none-any.whl (jupyter-events==0.12.0)
  Caused by: failed to create directory `c:\Program Files\Python313\Lib\site-packages\jupyter_events`: Accès refusé. (os error 5)


In [2]:
# =============================================================================
# IMPORTS
# =============================================================================

import warnings
warnings.filterwarnings("ignore")

# Manipulation de données
import numpy as np
import pandas as pd

# Manipulation d'images raster
import rasterio
from rasterio.windows import from_bounds
from rasterio.crs import CRS
from rasterio.warp import transform_bounds

# Accès à l'API Microsoft Planetary Computer
import pystac_client
import planetary_computer as pc

from tqdm import tqdm  # Barre de progression
import os

print("Imports OK!")

Imports OK!


---

## Étape 2 : Définition des constantes et fonctions

### Classes ESA WorldCover

On définit les classes et leurs noms pour les colonnes du CSV.

In [3]:
# =============================================================================
# CONSTANTES
# =============================================================================

# Classes ESA WorldCover (code → nom de la feature)
WORLDCOVER_CLASSES = {
    10: 'lc_tree',        # Tree cover
    20: 'lc_shrubland',   # Shrubland
    30: 'lc_grassland',   # Grassland
    40: 'lc_cropland',    # Cropland
    50: 'lc_builtup',     # Built-up
    60: 'lc_bare',        # Bare / sparse vegetation
    70: 'lc_snow',        # Snow and ice
    80: 'lc_water',       # Permanent water bodies
    90: 'lc_wetland',     # Herbaceous wetland
    95: 'lc_mangroves',   # Mangroves
    100: 'lc_moss',       # Moss and lichen
}

# Taille du buffer autour de chaque point (en mètres)
# 500m = on analyse l'occupation du sol dans un rayon de 500m autour du site
BUFFER_SIZE_METERS = 500

# Dossier de sortie
OUTPUT_DIR = "../data/processed"

print(f"Classes WorldCover : {list(WORLDCOVER_CLASSES.values())}")
print(f"Buffer : {BUFFER_SIZE_METERS}m autour de chaque point")

Classes WorldCover : ['lc_tree', 'lc_shrubland', 'lc_grassland', 'lc_cropland', 'lc_builtup', 'lc_bare', 'lc_snow', 'lc_water', 'lc_wetland', 'lc_mangroves', 'lc_moss']
Buffer : 500m autour de chaque point


### Fonction : Connexion au catalogue Planetary Computer

In [4]:
def get_worldcover_catalog():
    """
    Se connecte au catalogue Microsoft Planetary Computer et
    retourne l'accès à la collection ESA WorldCover.
    """
    catalog = pystac_client.Client.open(
        "https://planetarycomputer.microsoft.com/api/stac/v1",
        modifier=pc.sign_inplace,
    )
    print("Connexion au catalogue Planetary Computer OK!")
    return catalog

### Fonction : Calculer un buffer en degrés

Les coordonnées sont en degrés (lat/lon), mais on veut un buffer en mètres.
On utilise une approximation simple :
- 1 degré de latitude ≈ 111 km
- 1 degré de longitude ≈ 111 km × cos(latitude)

In [5]:
def meters_to_degrees(lat, meters):
    """
    Convertit une distance en mètres en degrés.
    
    Paramètres:
        lat : latitude du point (pour ajuster la longitude)
        meters : distance en mètres
    
    Retourne:
        (delta_lat, delta_lon) en degrés
    """
    # 1 degré de latitude ≈ 111 km
    delta_lat = meters / 111000
    
    # 1 degré de longitude dépend de la latitude
    delta_lon = meters / (111000 * np.cos(np.radians(lat)))
    
    return delta_lat, delta_lon

### Fonction : Extraire les pourcentages d'occupation du sol

Pour chaque site, on :
1. Trouve la tuile ESA WorldCover qui couvre le point
2. Découpe un buffer de 500m autour du point
3. Compte les pixels de chaque classe
4. Calcule le pourcentage de chaque classe

In [6]:
def extract_landcover_percentages(catalog, lat, lon, buffer_meters=500, debug=False):
    """
    Extrait les pourcentages de chaque classe d'occupation du sol
    dans un buffer autour d'un point.
    
    Paramètres:
        catalog : catalogue Planetary Computer connecté
        lat, lon : coordonnées du point
        buffer_meters : rayon du buffer en mètres
        debug : afficher les messages de débogage
    
    Retourne:
        dict avec le pourcentage de chaque classe (0-100)
    """
    # Initialiser les résultats à 0
    results = {name: 0.0 for name in WORLDCOVER_CLASSES.values()}
    
    try:
        # Calculer le buffer en degrés
        delta_lat, delta_lon = meters_to_degrees(lat, buffer_meters)
        
        # Bounding box autour du point
        bbox = [
            lon - delta_lon,  # min lon
            lat - delta_lat,  # min lat
            lon + delta_lon,  # max lon
            lat + delta_lat,  # max lat
        ]
        
        if debug:
            print(f"  Bbox: {bbox}")
        
        # Rechercher les tuiles ESA WorldCover qui couvrent ce point
        # Note: On enlève le filtre de version qui peut poser problème
        search = catalog.search(
            collections=["esa-worldcover"],
            bbox=bbox,
        )
        items = list(search.items())
        
        if debug:
            print(f"  Items trouvés: {len(items)}")
        
        if len(items) == 0:
            if debug:
                print("  ⚠️ Pas de données pour ce point")
            return results
        
        # Prendre le premier item (le plus récent)
        item = items[0]
        
        if debug:
            print(f"  Item: {item.id}")
            print(f"  Assets disponibles: {list(item.assets.keys())}")
        
        # Signer l'asset pour l'accès
        signed_asset = pc.sign(item.assets["map"])
        
        if debug:
            print(f"  URL signée: {signed_asset.href[:80]}...")
        
        # Ouvrir le raster et extraire la fenêtre
        with rasterio.open(signed_asset.href) as src:
            if debug:
                print(f"  CRS du raster: {src.crs}")
                print(f"  Bounds du raster: {src.bounds}")
            
            # Transformer la bbox dans le CRS du raster
            dst_crs = src.crs
            transformed_bbox = transform_bounds(
                CRS.from_epsg(4326),  # WGS84 (lat/lon)
                dst_crs,
                *bbox
            )
            
            if debug:
                print(f"  Bbox transformée: {transformed_bbox}")
            
            # Créer une fenêtre de lecture
            window = from_bounds(*transformed_bbox, src.transform)
            
            if debug:
                print(f"  Window: {window}")
            
            # Lire les données
            data = src.read(1, window=window)
            
            if debug:
                print(f"  Shape des données: {data.shape}")
                print(f"  Valeurs uniques: {np.unique(data)}")
            
            if data.size == 0:
                if debug:
                    print("  ⚠️ Données vides")
                return results
            
            # Compter les pixels de chaque classe
            total_pixels = data.size
            unique, counts = np.unique(data, return_counts=True)
            
            for class_code, class_name in WORLDCOVER_CLASSES.items():
                if class_code in unique:
                    idx = np.where(unique == class_code)[0][0]
                    results[class_name] = (counts[idx] / total_pixels) * 100
        
    except Exception as e:
        if debug:
            print(f"  ❌ Erreur: {type(e).__name__}: {e}")
        # En cas d'erreur, on retourne des 0
        pass
    
    return results

In [7]:
# =============================================================================
# TEST DE DIAGNOSTIC - Exécuter cette cellule pour voir pourquoi les données sont à 0
# =============================================================================

print("Test de diagnostic ESA WorldCover")
print("=" * 50)

# Se connecter au catalogue
catalog = get_worldcover_catalog()

# Tester avec un point en Afrique du Sud
test_lat, test_lon = -26.45, 28.085833  # Un point du dataset

print(f"\nTest avec le point: lat={test_lat}, lon={test_lon}")
print("-" * 50)

# Extraire avec debug activé
result = extract_landcover_percentages(catalog, test_lat, test_lon, 500, debug=True)

print("\nRésultats:")
for k, v in result.items():
    if v > 0:
        print(f"  {k}: {v:.1f}%")

Test de diagnostic ESA WorldCover
Connexion au catalogue Planetary Computer OK!

Test avec le point: lat=-26.45, lon=28.085833
--------------------------------------------------
  Bbox: [np.float64(28.08080185204243), -26.454504504504502, np.float64(28.09086414795757), -26.445495495495496]
  Items trouvés: 2
  Item: ESA_WorldCover_10m_2021_v200_S27E027
  Assets disponibles: ['map', 'input_quality', 'tilejson', 'rendered_preview']
  URL signée: https://ai4edataeuwest.blob.core.windows.net/esa-worldcover/v200/2021/map/ESA_Wo...
  CRS du raster: EPSG:4326
  Bounds du raster: BoundingBox(left=27.0, bottom=-27.0, right=30.0, top=-24.0)
  Bbox transformée: (28.08080185204243, -26.454504504504502, 28.09086414795757, -26.445495495495496)
  Window: Window(col_off=np.float64(12969.622224509167), row_off=np.float64(29345.945945945958), width=np.float64(120.74755098169044), height=np.float64(108.10810810807016))
  Shape des données: (108, 121)
  Valeurs uniques: [10 30 40 50 60 80]

Résultats:
  l

---

## Étape 3 : Extraction pour les données d'entraînement

On applique la fonction à tous les sites de mesure du dataset d'entraînement.

In [8]:
# Charger les données de qualité d'eau (nos sites de mesure)
Water_Quality_df = pd.read_csv("../data/raw/water_quality_training_dataset.csv")

print(f"Nombre de sites : {len(Water_Quality_df)}")
print(f"Colonnes : {list(Water_Quality_df.columns)}")
display(Water_Quality_df.head())

Nombre de sites : 9319
Colonnes : ['Latitude', 'Longitude', 'Sample Date', 'Total Alkalinity', 'Electrical Conductance', 'Dissolved Reactive Phosphorus']


,Latitude,Longitude,Sample Date,Total Alkalinity,Electrical Conductance,Dissolved Reactive Phosphorus
0,-28.760833,17.730278,02-01-2011,128.912,555.0,10.0
1,-26.861111,28.884722,03-01-2011,74.720,162.9,163.0
2,-26.450000,28.085833,03-01-2011,89.254,573.0,80.0
3,-27.671111,27.236944,03-01-2011,82.000,203.6,101.0
4,-27.356667,27.286389,03-01-2011,56.100,145.1,151.0


In [9]:
# Obtenir les sites uniques (lat/lon)
# On n'a pas besoin d'extraire plusieurs fois pour le même site
training_sites = Water_Quality_df[['Latitude', 'Longitude']].drop_duplicates().reset_index(drop=True)
print(f"Sites uniques à traiter : {len(training_sites)}")

Sites uniques à traiter : 162


In [10]:
# =============================================================================
# EXTRACTION ESA WORLDCOVER - TRAINING (avec sauvegarde incrémentale)
# =============================================================================

print("Connexion à Microsoft Planetary Computer...")
catalog = get_worldcover_catalog()

print(f"\nExtraction pour {len(training_sites)} sites uniques...")
print(f"Buffer : {BUFFER_SIZE_METERS}m autour de chaque point")

# Fichier de sauvegarde incrémentale
BACKUP_PATH = "../data/processed/worldcover_training_backup.csv"
SAVE_EVERY = 50  # Sauvegarder tous les 50 sites

print(f"Sauvegarde automatique tous les {SAVE_EVERY} sites → {BACKUP_PATH}\n")

# Liste pour stocker les résultats
training_results = []
completed_count = 0

for idx, row in tqdm(training_sites.iterrows(), total=len(training_sites), desc="Extraction"):
    lat, lon = row['Latitude'], row['Longitude']
    
    # Extraire les pourcentages
    lc_percentages = extract_landcover_percentages(catalog, lat, lon, BUFFER_SIZE_METERS)
    
    # Ajouter les coordonnées
    result = {'Latitude': lat, 'Longitude': lon}
    result.update(lc_percentages)
    
    training_results.append(result)
    completed_count += 1
    
    # Sauvegarde incrémentale
    if completed_count % SAVE_EVERY == 0:
        backup_df = pd.DataFrame(training_results)
        backup_df.to_csv(BACKUP_PATH, index=False)
        print(f"\n💾 Sauvegarde : {completed_count}/{len(training_sites)} sites")

# Sauvegarde finale
training_landcover_unique = pd.DataFrame(training_results)
training_landcover_unique.to_csv(BACKUP_PATH, index=False)

print(f"\n✅ Extraction terminée : {len(training_landcover_unique)} sites")

Connexion à Microsoft Planetary Computer...
Connexion au catalogue Planetary Computer OK!

Extraction pour 162 sites uniques...
Buffer : 500m autour de chaque point
Sauvegarde automatique tous les 50 sites → ../data/processed/worldcover_training_backup.csv



Extraction:  31%|███▏      | 51/162 [00:12<00:22,  5.03it/s]


💾 Sauvegarde : 50/162 sites


Extraction:  62%|██████▏   | 100/162 [00:24<00:16,  3.74it/s]


💾 Sauvegarde : 100/162 sites


Extraction:  93%|█████████▎| 150/162 [00:34<00:03,  3.48it/s]


💾 Sauvegarde : 150/162 sites


Extraction: 100%|██████████| 162/162 [00:37<00:00,  4.36it/s]


✅ Extraction terminée : 162 sites


In [11]:
# Fusionner avec le DataFrame original pour avoir une ligne par observation
# (plusieurs observations peuvent correspondre au même site à des dates différentes)
training_landcover_df = Water_Quality_df[['Latitude', 'Longitude', 'Sample Date']].merge(
    training_landcover_unique,
    on=['Latitude', 'Longitude'],
    how='left'
)

print(f"DataFrame final : {len(training_landcover_df)} lignes")

DataFrame final : 9319 lignes


In [12]:
# Sauvegarder le fichier CSV
output_path = os.path.join(OUTPUT_DIR, 'worldcover_features_training.csv')
training_landcover_df.to_csv(output_path, index=False)

print(f"Fichier créé : {output_path}")

Fichier créé : ../data/processed\worldcover_features_training.csv


In [13]:
# Aperçu du fichier créé
print("Aperçu des données extraites :")
print(f"- Lignes : {len(training_landcover_df)}")
print(f"- Colonnes : {list(training_landcover_df.columns)}")

# Colonnes d'occupation du sol
lc_cols = list(WORLDCOVER_CLASSES.values())

print(f"\nStatistiques des classes d'occupation du sol (% moyen) :")
stats = training_landcover_df[lc_cols].mean().sort_values(ascending=False)
for col, val in stats.items():
    if val > 0.1:  # Afficher seulement les classes > 0.1%
        print(f"  {col}: {val:.1f}%")

display(training_landcover_df.head())

Aperçu des données extraites :
- Lignes : 9319
- Colonnes : ['Latitude', 'Longitude', 'Sample Date', 'lc_tree', 'lc_shrubland', 'lc_grassland', 'lc_cropland', 'lc_builtup', 'lc_bare', 'lc_snow', 'lc_water', 'lc_wetland', 'lc_mangroves', 'lc_moss']

Statistiques des classes d'occupation du sol (% moyen) :
  lc_grassland: 39.5%
  lc_tree: 18.4%
  lc_shrubland: 16.1%
  lc_cropland: 15.9%
  lc_water: 4.1%
  lc_builtup: 4.0%
  lc_bare: 1.9%


,Latitude,Longitude,Sample Date,lc_tree,lc_shrubland,lc_grassland,lc_cropland,lc_builtup,lc_bare,lc_snow,lc_water,lc_wetland,lc_mangroves,lc_moss
0,-28.760833,17.730278,02-01-2011,9.131286,11.683228,0.910870,0.000000,0.000000,55.721168,0.0,22.553448,0.0,0.0,0.0
1,-26.861111,28.884722,03-01-2011,3.359351,0.038261,35.146924,61.072850,0.374962,0.000000,0.0,0.007652,0.0,0.0,0.0
2,-26.450000,28.085833,03-01-2011,2.816039,0.000000,47.520661,46.640649,1.469238,0.053566,0.0,1.499847,0.0,0.0,0.0
3,-27.671111,27.236944,03-01-2011,26.191560,0.091075,29.622040,4.401943,32.741348,0.159381,0.0,6.792653,0.0,0.0,0.0
4,-27.356667,27.286389,03-01-2011,5.297511,0.075896,70.332423,22.715543,1.555859,0.000000,0.0,0.022769,0.0,0.0,0.0


---

## Étape 4 : Extraction pour les données de validation

Même processus pour les données de **validation** (`submission_template.csv`).

In [14]:
# Charger le template de soumission (sites de validation)
Validation_df = pd.read_csv('../data/raw/submission_template.csv')

print(f"Nombre de sites de validation : {len(Validation_df)}")
display(Validation_df.head())

Nombre de sites de validation : 200


,Latitude,Longitude,Sample Date,Total Alkalinity,Electrical Conductance,Dissolved Reactive Phosphorus
0,-32.043333,27.822778,01-09-2014,NaN,NaN,NaN
1,-33.329167,26.077500,16-09-2015,NaN,NaN,NaN
2,-32.991639,27.640028,07-05-2015,NaN,NaN,NaN
3,-34.096389,24.439167,07-02-2012,NaN,NaN,NaN
4,-32.000556,28.581667,01-10-2014,NaN,NaN,NaN


In [15]:
# Obtenir les sites uniques
validation_sites = Validation_df[['Latitude', 'Longitude']].drop_duplicates().reset_index(drop=True)
print(f"Sites uniques à traiter : {len(validation_sites)}")

Sites uniques à traiter : 24


In [16]:
# =============================================================================
# EXTRACTION ESA WORLDCOVER - VALIDATION (avec sauvegarde incrémentale)
# =============================================================================

print(f"Extraction pour {len(validation_sites)} sites uniques...")

# Fichier de sauvegarde incrémentale
BACKUP_PATH_VAL = "../data/processed/worldcover_validation_backup.csv"
SAVE_EVERY_VAL = 20  # Sauvegarder tous les 20 sites

print(f"Sauvegarde automatique tous les {SAVE_EVERY_VAL} sites → {BACKUP_PATH_VAL}\n")

# Liste pour stocker les résultats
validation_results = []
completed_count = 0

for idx, row in tqdm(validation_sites.iterrows(), total=len(validation_sites), desc="Extraction"):
    lat, lon = row['Latitude'], row['Longitude']
    
    # Extraire les pourcentages
    lc_percentages = extract_landcover_percentages(catalog, lat, lon, BUFFER_SIZE_METERS)
    
    # Ajouter les coordonnées
    result = {'Latitude': lat, 'Longitude': lon}
    result.update(lc_percentages)
    
    validation_results.append(result)
    completed_count += 1
    
    # Sauvegarde incrémentale
    if completed_count % SAVE_EVERY_VAL == 0:
        backup_df = pd.DataFrame(validation_results)
        backup_df.to_csv(BACKUP_PATH_VAL, index=False)
        print(f"\n💾 Sauvegarde : {completed_count}/{len(validation_sites)} sites")

# Sauvegarde finale
validation_landcover_unique = pd.DataFrame(validation_results)
validation_landcover_unique.to_csv(BACKUP_PATH_VAL, index=False)

print(f"\n✅ Extraction terminée : {len(validation_landcover_unique)} sites")

Extraction pour 24 sites uniques...
Sauvegarde automatique tous les 20 sites → ../data/processed/worldcover_validation_backup.csv



Extraction:  83%|████████▎ | 20/24 [00:03<00:00,  6.13it/s]


💾 Sauvegarde : 20/24 sites


Extraction: 100%|██████████| 24/24 [00:04<00:00,  5.41it/s]


✅ Extraction terminée : 24 sites


In [17]:
# Fusionner avec le DataFrame original
validation_landcover_df = Validation_df[['Latitude', 'Longitude', 'Sample Date']].merge(
    validation_landcover_unique,
    on=['Latitude', 'Longitude'],
    how='left'
)

print(f"DataFrame final : {len(validation_landcover_df)} lignes")

DataFrame final : 200 lignes


In [18]:
# Sauvegarder le fichier CSV
output_path = os.path.join(OUTPUT_DIR, 'worldcover_features_validation.csv')
validation_landcover_df.to_csv(output_path, index=False)

print(f"Fichier créé : {output_path}")

Fichier créé : ../data/processed\worldcover_features_validation.csv


In [19]:
# Aperçu des données de validation
print(f"Données de validation : {len(validation_landcover_df)} lignes")
print(f"Colonnes : {list(validation_landcover_df.columns)}")

print(f"\nStatistiques des classes d'occupation du sol (% moyen) :")
stats = validation_landcover_df[lc_cols].mean().sort_values(ascending=False)
for col, val in stats.items():
    if val > 0.1:
        print(f"  {col}: {val:.1f}%")

display(validation_landcover_df.head())

Données de validation : 200 lignes
Colonnes : ['Latitude', 'Longitude', 'Sample Date', 'lc_tree', 'lc_shrubland', 'lc_grassland', 'lc_cropland', 'lc_builtup', 'lc_bare', 'lc_snow', 'lc_water', 'lc_wetland', 'lc_mangroves', 'lc_moss']

Statistiques des classes d'occupation du sol (% moyen) :
  lc_tree: 35.7%
  lc_grassland: 27.5%
  lc_shrubland: 21.4%
  lc_cropland: 9.8%
  lc_builtup: 2.5%
  lc_water: 1.6%
  lc_bare: 1.3%
  lc_wetland: 0.1%


,Latitude,Longitude,Sample Date,lc_tree,lc_shrubland,lc_grassland,lc_cropland,lc_builtup,lc_bare,lc_snow,lc_water,lc_wetland,lc_mangroves,lc_moss
0,-32.043333,27.822778,01-09-2014,6.394676,7.631655,70.392072,5.454282,6.604456,2.235243,0.0,1.287616,0.0,0.0,0.0
1,-33.329167,26.077500,16-09-2015,73.327591,22.968705,2.627046,0.000000,1.076658,0.000000,0.0,0.000000,0.0,0.0,0.0
2,-32.991639,27.640028,07-05-2015,61.663796,17.829457,17.915590,2.239449,0.100488,0.000000,0.0,0.251220,0.0,0.0,0.0
3,-34.096389,24.439167,07-02-2012,57.478089,6.184620,15.493356,20.794459,0.000000,0.049477,0.0,0.000000,0.0,0.0,0.0
4,-32.000556,28.581667,01-10-2014,36.395451,35.739283,11.628755,0.627005,0.021872,1.013415,0.0,14.574220,0.0,0.0,0.0


---

## Résumé

**Ce qu'on a fait :**
1. Connecté à Microsoft Planetary Computer
2. Pour chaque site de mesure :
   - Défini un buffer de 500m
   - Extrait la carte ESA WorldCover 2021
   - Calculé le pourcentage de chaque classe
3. Créé 2 fichiers CSV avec les features d'occupation du sol

**Features extraites :**

| Feature | Classe ESA WorldCover |
|---------|----------------------|
| lc_tree | Couvert forestier |
| lc_shrubland | Arbustes |
| lc_grassland | Prairies |
| lc_cropland | Zones agricoles |
| lc_builtup | Zones urbaines |
| lc_bare | Sol nu |
| lc_water | Plans d'eau |
| lc_wetland | Zones humides |

**Fichiers créés :**

| Fichier | Description |
|---------|-------------|
| worldcover_features_training.csv | Features pour l'entraînement |
| worldcover_features_validation.csv | Features pour la validation |

**Prochaine étape :**
- Intégrer ces features dans le modèle Random Forest
- Voir si l'occupation du sol améliore les prédictions